In [18]:
import requests
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# For normal

In [3]:
from gql import gql, Client
from gql.transport.aiohttp import AIOHTTPTransport

# Select your transport with a defined url endpoint
transport = AIOHTTPTransport(url="https://countries.trevorblades.com/")

# Create a GraphQL client using the defined transport
client = Client(transport=transport, fetch_schema_from_transport=True)

# Provide a GraphQL query


# Execute the query on the transport
# result = client.execute(query)
# print(result)


# Async for ipython

In [4]:
import asyncio

from gql import Client, gql
from gql.transport.aiohttp import AIOHTTPTransport

transport = AIOHTTPTransport(url="https://api.thegraph.com/subgraphs/name/connext/nxtp-matic")

## Graph Queries

### Transactions from User

In [5]:
query = gql(
    """
query GetSenderTransactions($userId: String!, $sendingChainId: BigInt!, $status: TransactionStatus) {
  transactions(
    where: {user: $userId, status: $status, sendingChainId: $sendingChainId}
    orderBy: preparedBlockNumber
    orderDirection: desc
  ) {
    id
    status
    chainId
    preparedTimestamp
    user {
      id
    }
    router {
      id
    }
    receivingChainTxManagerAddress
    sendingAssetId
    receivingAssetId
    sendingChainFallback
    receivingAddress
    callTo
    sendingChainId
    receivingChainId
    callDataHash
    transactionId
    amount
    expiry
    preparedBlockNumber
    bidSignature
    prepareCaller
    fulfillCaller
    cancelCaller
    prepareTransactionHash
    fulfillTransactionHash
    cancelTransactionHash
  }
}
"""
)

params = {
        "sendingChainId": 137,
        "userId": "0x4752a32843b661fedc525c979aaa01fa54d31366",
        "status": "Fulfilled"
    }

In [6]:
query = gql(
    """
query fetchAllTransactions($batch: Int!) {
  transactions (
    first:100, skip: $batch,
    orderBy: preparedBlockNumber
    orderDirection: asc
  ){
    id
    status
    chainId
    preparedTimestamp
    user {
      id
    }
    router {
      id
    }
    receivingChainTxManagerAddress
    sendingAssetId
    receivingAssetId
    sendingChainFallback
    receivingAddress
    callTo
    sendingChainId
    receivingChainId
    callDataHash
    transactionId
    amount
    expiry
    preparedBlockNumber
    bidSignature
    prepareCaller
    fulfillCaller
    cancelCaller
    prepareTransactionHash
    fulfillTransactionHash
    cancelTransactionHash
  }
}
"""
)

params = {
        "batch": 0
    }

## Querying

In [7]:
txn_columns = ['amount', 'bidSignature', 'callDataHash', 'callTo', 'cancelCaller', 'cancelTransactionHash', 'chainId', 'expiry', 'fulfillCaller', 'fulfillTransactionHash', 'id', 'prepareCaller', 'prepareTransactionHash', 'preparedBlockNumber', 'preparedTimestamp', 'receivingAddress', 'receivingAssetId', 'receivingChainId', 'receivingChainTxManagerAddress', 'router', 'sendingAssetId', 'sendingChainFallback', 'sendingChainId', 'status', 'transactionId', 'user']

matic_txns = pd.DataFrame(columns=txn_columns)
transport_matic = AIOHTTPTransport(url="https://api.thegraph.com/subgraphs/name/connext/nxtp-matic")

bsc_txns = pd.DataFrame(columns=txn_columns)
transport_bsc = AIOHTTPTransport(url="https://api.thegraph.com/subgraphs/name/connext/nxtp-bsc")

xdai_txns = pd.DataFrame(columns=txn_columns)
transport_xdai = AIOHTTPTransport(url="https://api.thegraph.com/subgraphs/name/connext/nxtp-xdai")

fantom_txns = pd.DataFrame(columns=txn_columns)
transport_fantom = AIOHTTPTransport(url="https://api.thegraph.com/subgraphs/name/connext/nxtp-fantom")

In [8]:
async def graph_query(query, params, transport, transactions):

    # Using `async with` on the client will start a connection on the transport
    # and provide a `session` variable to execute queries on this connection
    async with Client(
        transport=transport, fetch_schema_from_transport=True,
    ) as session:

        # Execute single query
        for batch in range(51):
            result = await session.execute(query, variable_values=params)
            for tr in result["transactions"]:
                list_values = list(tr.values())
                list_values[19] = list_values[19]["id"]
                list_values[25] = list_values[25]["id"]
                transactions.loc[len(transactions.index)] = list_values
            if len(result["transactions"]) < 100:
                break
            params['batch']+=100
            print(params["batch"], end="-")
        print("")
        return result

In [9]:
params["batch"] = 0
await graph_query(query, params, transport_matic, matic_txns)
params["batch"] = 0
await graph_query(query, params, transport_bsc, bsc_txns)
params["batch"] = 0
await graph_query(query, params, transport_xdai, xdai_txns)
params["batch"] = 0
await graph_query(query, params, transport_fantom, fantom_txns)
params["batch"] = 0

100-200-300-400-500-600-700-800-900-1000-1100-1200-1300-1400-1500-1600-1700-1800-1900-2000-2100-2200-2300-2400-2500-2600-2700-2800-2900-3000-3100-3200-3300-3400-3500-3600-3700-3800-3900-4000-4100-4200-4300-4400-4500-4600-
100-200-300-400-500-600-700-800-900-1000-1100-1200-1300-1400-1500-1600-1700-1800-1900-2000-2100-2200-2300-2400-2500-2600-2700-2800-2900-3000-3100-3200-3300-3400-3500-3600-
100-200-300-400-500-600-
100-200-300-400-500-600-700-800-900-1000-1100-1200-1300-


In [21]:
matic_txns[matic_txns.sendingChainId=="137"].shape

(3156, 26)

In [25]:
bsc_txns[bsc_txns.sendingChainId=="56"].shape

(1336, 26)

In [26]:
xdai_txns[xdai_txns.sendingChainId=="100"].shape

(325, 26)

In [27]:
fantom_txns[fantom_txns.sendingChainId=="250"].shape

(334, 26)

In [28]:
3156+1336+325+334

5151

## from and to chains

BSC : 56
<br>
xDai : 100
<br>
Polygon : 137
<br>
Fantom : 250
<br>

In [ ]:
result = pd.concat([matic_txns,bsc_txns,xdai_txns,fantom_txns])

In [39]:
matic_txns["receivingAddress"].value_counts()

0x9381aff046755229b398c5be1d4fab64d628d008    49
0x7a12f8072667a18092e90fc6485d468caaa2179f    26
0x68b7d230072090924bb254650cf9df3dc1089745    23
0x56c9656662fad432973e8743e1f6d2a79df61a35    22
0x34a500632be5498a463fadf5be7258c1cf7bf52b    20
                                              ..
0x614ee992a7db0dd1eea121f32f1cc8387ddc7fd5     1
0xf1d918c34c71fc121e05c53e934d431af3ce4252     1
0x7cbcc70f5176e54d72476b8a60e35fe7ebaff113     1
0xdc9cdd31bc0f598e6b2c8302312b40852f636e60     1
0x11c641a959e703f3130a7b70a33a0fec814ff486     1
Name: receivingAddress, Length: 2772, dtype: int64

In [41]:
matic_txns[matic_txns["receivingAddress"] == "0x4752a32843b661fedc525c979aaa01fa54d31366"]

,amount,bidSignature,callDataHash,callTo,cancelCaller,cancelTransactionHash,chainId,expiry,fulfillCaller,fulfillTransactionHash,id,prepareCaller,prepareTransactionHash,preparedBlockNumber,preparedTimestamp,receivingAddress,receivingAssetId,receivingChainId,receivingChainTxManagerAddress,router,sendingAssetId,sendingChainFallback,sendingChainId,status,transactionId,user
3029,200000000000000000,0x7c2a8087577e476fed214ec6c76179f6ff803847d436...,0xc5d2460186f7233c927e7db2dcc703c0e500b653ca82...,0x0000000000000000000000000000000000000000,None,None,137,1631189000,0xe1dd7b28ab085978ead65aff92b22b6322a96422,0xb1c8c92a707d7e3229487282bb90a676104533bb4e83...,0x144c7d9570851fc0c053150125079edd156a6b4ea53f...,0x4752a32843b661fedc525c979aaa01fa54d31366,0x64f1e20b58adffee60a52cc113aec275427f5ced7d30...,18809304,1630931002,0x4752a32843b661fedc525c979aaa01fa54d31366,0x0000000000000000000000000000000000000000,100,0x31efc4aeaa7c39e54a33fdc3c46ee2bd70ae0a09,0xe1dd7b28ab085978ead65aff92b22b6322a96422,0x8f3cf7ad23cd3cadbd9735aff958023239c6a063,0x4752a32843b661fedc525c979aaa01fa54d31366,137,Fulfilled,0x144c7d9570851fc0c053150125079edd156a6b4ea53f...,0x4752a32843b661fedc525c979aaa01fa54d31366
3088,200000000000000000,0x3986b12bda7f56909a130f5c677fc57f4e9a08c813c7...,0xc5d2460186f7233c927e7db2dcc703c0e500b653ca82...,0x0000000000000000000000000000000000000000,None,None,137,1631193933,0xe1dd7b28ab085978ead65aff92b22b6322a96422,0x8ca1a4e9d48b3b6d1b49dc411d303741b13512cc62ac...,0x099f2f7b4800eaa392dd017cee8145968b983e6ad712...,0x4752a32843b661fedc525c979aaa01fa54d31366,0xb7825ea3b34f8923eb9252dbcd16952e2580797fe671...,18811078,1630934800,0x4752a32843b661fedc525c979aaa01fa54d31366,0x0000000000000000000000000000000000000000,100,0x31efc4aeaa7c39e54a33fdc3c46ee2bd70ae0a09,0xe1dd7b28ab085978ead65aff92b22b6322a96422,0x8f3cf7ad23cd3cadbd9735aff958023239c6a063,0x4752a32843b661fedc525c979aaa01fa54d31366,137,Fulfilled,0x099f2f7b4800eaa392dd017cee8145968b983e6ad712...,0x4752a32843b661fedc525c979aaa01fa54d31366


In [42]:
matic_txns["receivingAssetId"].value_counts()

0x8ac76a51cc950d9822d68b83fe1ad97b32cd580d    1226
0x55d398326f99059ff775485246999027b3197955     754
0x2791bca1f2de4661ed88a30c99a7a9449aa84174     730
0xc2132d05d31c914a87c6611c10748aeb04b58e8f     497
0x04068da6c83afcfa0e13ba15a6696662335d5b75     454
0x8f3cf7ad23cd3cadbd9735aff958023239c6a063     242
0x8d11ec38a3eb5e956b052f67da8bdc9bef8abf3e     186
0x1af3f329e8be154074d8769d1ffa4ee058b1dbc3     180
0x0000000000000000000000000000000000000000     150
0x049d68029688eabf473097a2fc38ef61633a3c7a     105
0xddafbb505ad214d7b80b1f830fccc89b60fb7a83      79
0x4ecaba5870353805a9f068101a40e0f32ed605c6      22
Name: receivingAssetId, dtype: int64

In [43]:
chain_mapping = {"56": "BSC", "137": "Polygon", "250": "Fantom", "100": "xDai"}

In [44]:
def transacting_chains(row):
    val = chain_mapping[row["sendingChainId"]] + " -> " + chain_mapping[row["receivingChainId"]]
    return val

In [47]:
matic_txns['asset_movement'] = matic_txns.apply(transacting_chains, axis=1)

matic_txns['asset_movement'].value_counts()

Polygon -> BSC       2160
BSC -> Polygon       1073
Polygon -> Fantom     745
Polygon -> xDai       251
xDai -> Polygon       209
Fantom -> Polygon     187
Name: asset_movement, dtype: int64

In [49]:
matic_txns['asset_movement'] = matic_txns.apply(transacting_chains, axis=1)
matic_txns['asset_movement'].value_counts()

Polygon -> BSC       2160
BSC -> Polygon       1073
Polygon -> Fantom     745
Polygon -> xDai       251
xDai -> Polygon       209
Fantom -> Polygon     187
Name: asset_movement, dtype: int64

In [50]:
matic_txns[matic_txns["status"]=="Prepared"]['asset_movement'].value_counts()

BSC -> Polygon       122
Polygon -> Fantom     21
Fantom -> Polygon     12
Name: asset_movement, dtype: int64

In [48]:
bsc_txns['asset_movement'] = bsc_txns.apply(transacting_chains, axis=1)

bsc_txns['asset_movement'].value_counts()

Polygon -> BSC    2124
BSC -> Polygon    1076
BSC -> Fantom      213
Fantom -> BSC      113
xDai -> BSC         55
BSC -> xDai         47
Name: asset_movement, dtype: int64